# Zonotope Containment: Empirial Study of Necessity Gaps

In [1]:
import numpy as np
import pypolycontain as pp
import time
import pydrake.solvers.mathematicalprogram as MP
# use Gurobi solver
import pydrake.solvers.gurobi as Gurobi_drake
gurobi_solver=Gurobi_drake.GurobiSolver()
np.random.seed(0)

In [2]:
def alpha_necessary_old(Z_i,Z_o):
    program=MP.MathematicalProgram()
    zeta=program.NewContinuousVariables(Z_o.G.shape[1],2**Z_i.G.shape[1],"zeta")
    beta=program.NewContinuousVariables(1,"beta")
    V=pp.vcube(Z_i.G.shape[1])
    for i in range(V.shape[0]):
        program.AddLinearEqualityConstraint( Aeq=Z_o.G, beq=np.dot(Z_i.G,V.T)[:,i], vars= zeta[:,i]  )
    program.AddLinearConstraint(  np.less_equal(zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )
    program.AddLinearConstraint(  np.less_equal(-zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )
    program.AddLinearCost(np.eye(1),np.zeros((1)),beta)
    result=gurobi_solver.Solve(program,None,None)
    if result.is_success():
        alpha=1/result.GetSolution(beta)[0]
        return alpha
    else:
        print("optimization failed")
        
def alpha_necessary_older(Z_i,Z_o):
    alpha_min=np.inf
    V=pp.vcube(Z_i.G.shape[1])
    B=np.dot(Z_i.G,V.T)
    for i in range(V.shape[0]):
        program=MP.MathematicalProgram()
        zeta=program.NewContinuousVariables(Z_o.G.shape[1],"zeta")
        beta=program.NewContinuousVariables(1,"beta")
        program.AddLinearEqualityConstraint( Aeq=Z_o.G, beq=B[:,i:i+1], vars= zeta  )
#         for j in range(zeta.shape[0]):
#             program.AddLinearConstraint(  zeta[j]-beta, -np.inf, 0 )
#             program.AddLinearConstraint(  -zeta[j]+ beta, -np.inf, 0 )
        program.AddLinearConstraint(  np.less_equal(zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )
        program.AddLinearConstraint(  np.less_equal(-zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )
        program.AddLinearCost(np.eye(1),np.zeros((1)),beta)
        result=gurobi_solver.Solve(program,None,None)
        if result.is_success():
            alpha=1/result.GetSolution(beta)[0]
            alpha_min=min(alpha,alpha_min)
        else:
            print("optimization failed")
    return alpha_min

def alpha_necessary(Z_i,Z_o):
    alpha_min=np.inf
    V=pp.vcube(Z_i.G.shape[1])
    B=np.dot(Z_i.G,V.T)
    for i in range(V.shape[0]):
        program=MP.MathematicalProgram()
        zeta=program.NewContinuousVariables(Z_o.G.shape[1],"zeta")
        alpha=program.NewContinuousVariables(1,"alpha")
        Aeq=np.hstack(( Z_o.G, -B[:,i:i+1]))
        program.AddLinearEqualityConstraint( Aeq=np.hstack(( Z_o.G, -B[:,i:i+1])), \
                                            beq=np.zeros((Z_o.G.shape[0])),\
                                            vars= np.hstack((zeta,alpha))  )
        program.AddBoundingBoxConstraint(-1,1,zeta)
        program.AddLinearCost(-np.eye(1),np.zeros((1)),alpha)
        result=gurobi_solver.Solve(program,None,None)
        if result.is_success():
            alpha=result.GetSolution(alpha)[0]
            alpha_min=min(alpha,alpha_min)
        else:
            print("optimization failed")
    return alpha_min
        
        
def alpha_sufficient(Z_i,Z_o):
    program=MP.MathematicalProgram()
    beta=program.NewContinuousVariables(1,"beta")
    circumbody=pp.to_AH_polytope(Z_o)
    parametric_circumbody=circumbody.copy()
    parametric_circumbody.P.h=circumbody.P.h*beta
    Theta,*_=pp.subset(program,Z_i,parametric_circumbody,k=-1)
    program.AddLinearCost(np.eye(1),np.zeros((1)),beta)
    result=gurobi_solver.Solve(program,None,None)
    if result.is_success():
        alpha=1/result.GetSolution(beta)[0]
        return alpha
    else:
        print("optimization failed")

In [3]:
N,n_max=100,20
D=10 # Percent to report
k,j=0,0 # Counter
d,j={},0
Q_o={ 3:[5,10,15,20], 5:[10,15,20], 10:[15,20], 15:[20] }
for n in Q_o.keys():
    print("="*20,"\n","n=",n)
    for q_o in Q_o[n]:
        d[n,q_o]=np.zeros(N)
        for i in range(N):
            j+=1
            print(n,q_o,i)
#             q_i=np.random.randint(n,q_o+1)
            z_i=pp.zonotope(G=np.eye(n))
            z_o=pp.zonotope(G=np.random.normal(size=(n,q_o)))
#             x=time.time()
            a_suf=alpha_sufficient(z_i,z_o)
#             print("sufficient:",time.time()-x)
#             x=time.time()
            a_nec=alpha_necessary(z_i,z_o)
#             print("necessary:",time.time()-x)
            d[n,q_o][i]=1-a_suf/a_nec
#             print("gap:",n,q_o,"=",d[n,q_o][i])
            j+=1
print("Done")

 n= 3
3 5 0
3 5 1
3 5 2
3 5 3
3 5 4
3 5 5
3 5 6
3 5 7
3 5 8
3 5 9
3 5 10
3 5 11
3 5 12
3 5 13
3 5 14
3 5 15
3 5 16
3 5 17
3 5 18
3 5 19
3 5 20
3 5 21
3 5 22
3 5 23
3 5 24
3 5 25
3 5 26
3 5 27
3 5 28
3 5 29
3 5 30
3 5 31
3 5 32
3 5 33
3 5 34
3 5 35
3 5 36
3 5 37
3 5 38
3 5 39
3 5 40
3 5 41
3 5 42
3 5 43
3 5 44
3 5 45
3 5 46
3 5 47
3 5 48
3 5 49
3 5 50
3 5 51
3 5 52
3 5 53
3 5 54
3 5 55
3 5 56
3 5 57
3 5 58
3 5 59
3 5 60
3 5 61
3 5 62
3 5 63
3 5 64
3 5 65
3 5 66
3 5 67
3 5 68
3 5 69
3 5 70
3 5 71
3 5 72
3 5 73
3 5 74
3 5 75
3 5 76
3 5 77
3 5 78
3 5 79
3 5 80
3 5 81
3 5 82
3 5 83
3 5 84
3 5 85
3 5 86
3 5 87
3 5 88
3 5 89
3 5 90
3 5 91
3 5 92
3 5 93
3 5 94
3 5 95
3 5 96
3 5 97
3 5 98
3 5 99
3 10 0
3 10 1
3 10 2
3 10 3
3 10 4
3 10 5
3 10 6
3 10 7
3 10 8
3 10 9
3 10 10
3 10 11
3 10 12
3 10 13
3 10 14
3 10 15
3 10 16
3 10 17
3 10 18
3 10 19
3 10 20
3 10 21
3 10 22
3 10 23
3 10 24
3 10 25
3 10 26
3 10 27
3 10 28
3 10 29
3 10 30
3 10 31
3 10 32
3 10 33
3 10 34
3 10 35
3 10 36
3 10 37
3 10 38
3 

15 20 98
15 20 99
Done


In [4]:
for key in d.keys():
    print(key[0],' & ',key[1],' & ',np.sum(d[key]<0.001), ' & ', np.sum(d[key]<0.01), ' & ',\
         np.sum(d[key]<0.05), ' & ', np.sum(d[key]<0.05), np.round(np.max(d[key]),3) , ' \\\ \\hline')

3  &  5  &  97  &  97  &  100  &  0.039  \\ \hline
3  &  10  &  89  &  94  &  99  &  0.051  \\ \hline
3  &  15  &  94  &  96  &  100  &  0.041  \\ \hline
3  &  20  &  94  &  98  &  100  &  0.024  \\ \hline
5  &  10  &  82  &  86  &  96  &  0.086  \\ \hline
5  &  15  &  77  &  81  &  98  &  0.077  \\ \hline
5  &  20  &  60  &  68  &  92  &  0.104  \\ \hline
10  &  15  &  66  &  76  &  96  &  0.095  \\ \hline
10  &  20  &  29  &  41  &  70  &  0.177  \\ \hline
15  &  20  &  58  &  71  &  94  &  0.1  \\ \hline
